In [15]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"]= GROQ_API_KEY

In [17]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="llama-3.3-70b-versatile")
llm.invoke("hi").content

"It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [18]:
class Chatbot:
    def __init__(self,system=""):
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})
    def __call__(self,message):
        self.message.append({"role":"user","content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
        
    def execute(self):
        llm = ChatGroq(model_name="Gemma2-9b-It")
        result = llm.invoke(self.message)
        return result.content
        

In [19]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

In [20]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [21]:
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]
def calculate(number):
    return eval(number)

In [22]:
def calculate(number):
    return eval(number)

In [23]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [24]:
import re
action_re = re.compile('^Action: (\w+): (.*)')

In [25]:

def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result
    

In [26]:
query("Fifteen * twenty five")

Thought: Let's calculate 15 * 25. 
Action: calculate: 15 * 25
PAUSE 

 -- running calculate 15 * 25
Observation: 375
Answer: 15 * 25 = 375  



'Answer: 15 * 25 = 375  \n'

In [27]:
query("what is AI") 

Thought: I need to find a good definition of AI for my answer. I think searching Simon's blog would be a good start.

Action: simon_blog_search: AI 
PAUSE 

 -- running simon_blog_search AI 
Observation: It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomplete."
That strawman is right: it's not "intelligent" in the same way that humans are. And "spicy autocomplet

'Thought: Wikipedia seems to be cutting off.  I should probably try a different approach. I think I have enough information to formulate an answer. \n\nAnswer: AI, or Artificial Intelligence, refers to the ability of machines, especially computers, to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. It is a rapidly developing field of research with the goal of creating systems that can mimic human cognitive abilities. \n'

In [28]:

next_prompt="Tell me about the current gdp of PAKISATN  from Wikipedia."

In [29]:
query("Has Simon written about AI?")

Thought: I need to check Simon's blog for information about AI.

Action: simon_blog_search: AI
PAUSE 

 -- running simon_blog_search AI
Observation: It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomplete."
That strawman is right: it's not "intelligent" in the same way that humans are. And "spicy autocomplete" is actually a pretty good analogy for how a lot of t

'Thought: Simon updated his blog post about AI in 2024. \n\nAnswer: Yes, Simon has written about AI. \n'

In [30]:
print("'Answer: Yes, Simon has written about AI. \n'")

'Answer: Yes, Simon has written about AI. 
'


In [33]:
query("can you tell me capital of usa?")

Thought: I should search for the capital of the USA on Wikipedia. 

Action: wikipedia: United States 
PAUSE  

 -- running wikipedia United States 
Observation: The <span class="searchmatch">United</span> <span class="searchmatch">States</span> of America (USA), commonly known as the <span class="searchmatch">United</span> <span class="searchmatch">States</span> (U.S.) or America, is a country primarily located in North America. It is a federal
Thought: The observation mentions the US is located in North America and is a country, but I need to find its capital. 

Action: wikipedia: United States 
PAUSE  

 -- running wikipedia United States 
Observation: The <span class="searchmatch">United</span> <span class="searchmatch">States</span> of America (USA), commonly known as the <span class="searchmatch">United</span> <span class="searchmatch">States</span> (U.S.) or America, is a country primarily located in North America. It is a federal
Thought: I need to keep looking for the capital c